In [ ]:
# Install the boto library.
import ibm_boto3
from ibm_botocore.client import Config
import os

In [ ]:
cos_credentials = {
  "apikey": "*******",
  "cos_hmac_keys": {
    "access_key_id": "******",
    "secret_access_key": "******"
  },
  "endpoints": "https://cos-service.bluemix.net/endpoints",
  "iam_apikey_description": "*******",
  "iam_apikey_name": "*******",
  "iam_role_crn": "*******",
  "iam_serviceid_crn": "*******",
  "resource_instance_id": "*******"
}

In [ ]:
service_endpoint = 'https://s3-api.us-geo.objectstorage.softlayer.net'
auth_endpoint = 'https://iam.bluemix.net/oidc/token'

In [ ]:
# Create a COS resource.
cos = ibm_boto3.resource('s3',
                         ibm_api_key_id=cos_credentials['apikey'],
                         ibm_service_instance_id=cos_credentials['resource_instance_id'],
                         ibm_auth_endpoint=auth_endpoint,
                         config=Config(signature_version='oauth'),
                         endpoint_url=service_endpoint)

In [ ]:
from uuid import uuid4

bucket_uid = str(uuid4())
buckets = ['training-data-' + bucket_uid, 'training-results-' + bucket_uid]

for bucket in buckets:
    if not cos.Bucket(bucket) in cos.buckets.all():
        print('Creating bucket "{}"...'.format(bucket))
        try:
            cos.create_bucket(Bucket=bucket)
        except ibm_boto3.exceptions.ibm_botocore.client.ClientError as e:
            print('Error: {}.'.format(e.response['Error']['Message']))

In [ ]:
# Display a list of created buckets.
print(list(cos.buckets.all()))

In [ ]:
file_dirs = "dataset/"
file_dirs = [file_dirs + i for i in os.listdir(file_dirs)]
file_dirs

In [ ]:
# Upload files to COS

bucket_obj = cos.Bucket(buckets[0])

for file_dir in file_dirs:
    filename = file_dir.split('/')[-1]
    print('Uploading data {}...'.format(filename))
    with open(file_dir, 'rb') as f:
        bucket_obj.upload_fileobj(f, filename)
        print('{} is uploaded.'.format(filename))

In [ ]:
import urllib3, requests, json, base64, time, os, wget

In [ ]:
wml_credentials = {
  "apikey": "*******",
  "iam_apikey_description": "*******",
  "iam_apikey_name": "*******",
  "iam_role_crn": "*******",
  "iam_serviceid_crn": "*******",
  "instance_id": "*******",
  "password": "*******",
  "url": "https://us-south.ml.cloud.ibm.com",
  "username": "*******"
}

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [ ]:
client = WatsonMachineLearningAPIClient(wml_credentials)
print(client.version)

In [ ]:
model_definition_metadata = {
            client.repository.DefinitionMetaNames.NAME: "QANet",
            client.repository.DefinitionMetaNames.DESCRIPTION: "training Q&A NET",
            client.repository.DefinitionMetaNames.AUTHOR_NAME: "Owen",
            client.repository.DefinitionMetaNames.FRAMEWORK_NAME: "tensorflow",
            client.repository.DefinitionMetaNames.FRAMEWORK_VERSION: "1.5",
            client.repository.DefinitionMetaNames.RUNTIME_NAME: "python",
            client.repository.DefinitionMetaNames.RUNTIME_VERSION: "3.6",
            client.repository.DefinitionMetaNames.EXECUTION_COMMAND: "python3 training.py"
            }

In [ ]:
filename = "QA_system.zip"
definition_details = client.repository.store_definition(filename, model_definition_metadata)
definition_uid = client.repository.get_definition_uid(definition_details)
print(definition_uid)

In [ ]:
# Configure the training metadata for the TRAINING_DATA_REFERENCE and TRAINING_RESULTS_REFERENCE.
training_configuration_metadata = {
            client.training.ConfigurationMetaNames.NAME: "QANet", 
            client.training.ConfigurationMetaNames.AUTHOR_NAME: "Owen Zhu",              
            client.training.ConfigurationMetaNames.DESCRIPTION: "QANet Training",
            client.training.ConfigurationMetaNames.COMPUTE_CONFIGURATION: {"name": "k80"},
            client.training.ConfigurationMetaNames.TRAINING_DATA_REFERENCE: {
                    "connection": {
                        "endpoint_url": service_endpoint,
                        "aws_access_key_id": cos_credentials['cos_hmac_keys']['access_key_id'],
                        "aws_secret_access_key": cos_credentials['cos_hmac_keys']['secret_access_key']
                    },
                    "source": {
                        "bucket": buckets[0],
                    },
                    "type": "s3"
                },
            client.training.ConfigurationMetaNames.TRAINING_RESULTS_REFERENCE: {
                "connection": {
                    "endpoint_url": service_endpoint,
                    "aws_access_key_id": cos_credentials['cos_hmac_keys']['access_key_id'],
                    "aws_secret_access_key": cos_credentials['cos_hmac_keys']['secret_access_key']
                },
                "target": {
                    "bucket": buckets[1],
                },
                "type": "s3"
            },
        }

In [ ]:
training_run_details = client.training.run(definition_uid, training_configuration_metadata)

In [ ]:
training_run_guid_async = client.training.get_run_uid(training_run_details)

In [ ]:
# Get training run status.
status = client.training.get_status(training_run_guid_async)
print(json.dumps(status, indent=2))